<a href="https://colab.research.google.com/github/soumabkargit/intro-to-machine-learning/blob/master/istic_m2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install pandas_ods_reader

import tensorflow as tf
from pandas_ods_reader import read_ods
import shutil
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
print(tf.__version__)

# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download du fichier
id = '1QI-6i1pQgyXpGZPWKKYKdKHIX9nGvkN0'
downloaded = drive.CreateFile({'id': '1QI-6i1pQgyXpGZPWKKYKdKHIX9nGvkN0'})
downloaded.GetContentFile('subjective_test_scores.ods')

# Path of the file to read
data = 'subjective_test_scores.ods'

# load a sheet based on its index (1 based)
sheet_idx = 1
df_all_data = read_ods(data, sheet_idx, columns=['QP', 'Davg', 'Dmax', 'N' , 'MOS'])
df_train = df_all_data[1:81]
df_valid = df_all_data[82:98]
df_test = df_all_data[99:116]

CSV_COLUMN_NAMES = list(df_all_data)
print(CSV_COLUMN_NAMES)

FEATURE_NAMES = CSV_COLUMN_NAMES[0:3] # 
LABEL_NAME = CSV_COLUMN_NAMES[4] #

#Create feature columns

feature_columns = [tf.feature_column.numeric_column(key = k) for k in FEATURE_NAMES]

#Define input function

def train_input_fn(df, batch_size = 128):
    #1. Convert dataframe into correct (features,label) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(tensors = (dict(df[FEATURE_NAMES]), df[LABEL_NAME]))
    
    # Note:
    # If we returned now, the Dataset would iterate over the data once  
    # in a fixed order, and only produce a single element at a time.
    
    #2. Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(buffer_size = 1000).repeat(count = None).batch(batch_size = batch_size)
   
    return dataset


def eval_input_fn(df, batch_size = 128):
    #1. Convert dataframe into correct (features,label) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(tensors = (dict(df[FEATURE_NAMES]), df[LABEL_NAME]))
    
    #2.Batch the examples.
    dataset = dataset.batch(batch_size = batch_size)
   
    return dataset


def predict_input_fn(df, batch_size = 128):
    #1. Convert dataframe into correct (features) format for Estimator API
    dataset = tf.data.Dataset.from_tensor_slices(tensors = dict(df[FEATURE_NAMES])) # no label

    #2.Batch the examples.
    dataset = dataset.batch(batch_size = batch_size)
   
    return dataset

#Choose Estimator
OUTDIR = "QoS"

model = tf.estimator.LinearRegressor(
    feature_columns = feature_columns,
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed = 1) # for reproducibility
)

#Train

tf.logging.set_verbosity(tf.logging.INFO) # so loss is printed during training
shutil.rmtree(path = OUTDIR, ignore_errors = True) # start fresh each time

model.train(
    input_fn = lambda: train_input_fn(df = df_train), 
    steps = 500)

def print_rmse(model, df):
    metrics = model.evaluate(input_fn = lambda: eval_input_fn(df))
    print("RMSE on dataset = {}".format(metrics["average_loss"]**.5))
print_rmse(model = model, df = df_valid)

#Predict
predictions = model.predict(input_fn = lambda: predict_input_fn(df = df_test))
for items in predictions:
    print(items)


#Change Estimator type


tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(path = OUTDIR, ignore_errors = True) 
model = tf.estimator.DNNRegressor(
    hidden_units = [10,10], # specify neural architecture
    feature_columns = feature_columns, 
    model_dir = OUTDIR,
    config = tf.estimator.RunConfig(tf_random_seed = 1)
)
model.train(
    input_fn = lambda: train_input_fn(df = df_train), 
    steps = 500)
print_rmse(model = model, df = df_valid)





1.15.0
['QP', 'Davg', 'Dmax', 'N', 'MOS']
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe4e2b89b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done call